In [1]:
import taichi as ti
import pyvista as pv
import numpy as np
from pyvista import CellType
ti.init(ti.cpu)

[Taichi] version 1.7.3, llvm 15.0.1, commit 5ec301be, win, python 3.12.4
[Taichi] Starting on arch=x64


In [2]:
class Assembly():
    def __init__(self,meshes:dict[str,pv.PolyData]|list[pv.PolyData]|tuple[pv.PolyData]|pv.PolyData,mesh_specification = 'euler') -> None:
        self.mesh_specification = mesh_specification
        self.meshes = {}
        
        self.add_meshes(meshes)
        self.groups = {}

    def add_meshes(self,meshes):
        if isinstance(meshes,dict):
            for name,mesh in meshes.items():
                self.add_mesh(mesh,name)
        elif isinstance(meshes,(list,tuple)):
            for i,mesh in enumerate(meshes):
                self.add_mesh(mesh,f'mesh_{i}')
        elif isinstance(meshes,pv.PolyData):
                i = len(self.meshes.values())
                self.add_mesh(meshes,f'mesh_{i}')

    def add_mesh(self,mesh:pv.PolyData,name:str):
        assert isinstance(mesh,pv.PolyData)
        name = f'{name}-1' if name in self.meshes.keys() else name
        self.meshes[name] = mesh 




In [3]:
from src.preprocess import Mesh

In [4]:
VALID_FP_DTYPES = {ti.f32,ti.f16,ti.f64}
def create_field_output_stuct(*,return_grad = False,**kwargs) -> ti.Struct:
    if not kwargs:
        kwargs = {'u':ti.math.vec3,'p':ti.f32}
    
    field_struct = ti.types.struct(**kwargs)


    if return_grad:
        gradients = {}
        for key,val in kwargs.items():
            name = f'grad_{key}'
            if hasattr(val,'n'):
                gradients[name] =  ti.types.matrix(val.n,3,dtype=val.dtype)
            else:
                assert val in VALID_FP_DTYPES
                gradients[name] = ti.types.matrix(1,3,dtype=val)
                

        gradient_struct = ti.types.struct(**gradients)
        return field_struct,gradient_struct



    return field_struct

fields = create_field_output_stuct(return_grad=True)


In [5]:
fields[1].members['grad_u'].n

3

In [6]:
x = fields[0](u=[1,1,1],p = 1)

x['u'][0:2]

array([1., 1.], dtype=float32)

In [7]:

def create_FVM_cell(num_nodes:int,num_faces:int,float_dtype = ti.f32):
    face_array_ids = ti.types.vector(num_faces,int)
    face_array_float = ti.types.vector(num_faces,float_dtype)
    face_array_vector = ti.types.matrix(num_faces,3,float_dtype)

    node_array_ids = ti.types.vector(num_nodes,dtype=float_dtype)
     
    centroid_vector = ti.types.vector(3,dtype=float_dtype)


    @ti.dataclass
    class Cell():
        '''
        Stores predefined information such as neighbors, nodes, face normals etc into a convienent struct
        '''
        id:int
        nodes: node_array_ids
        centroid:centroid_vector # (3,)
        faces: face_array_ids
        face_normals: face_array_vector
        faces_area: face_array_float
        neighbor_ids: face_array_ids # Use -1 for no neighbor
        neighbor_distance: face_array_float
        volume:float_dtype
    return Cell



c = create_FVM_cell(8,6)


a = c()




In [8]:
    

a= ti.math.mat3([[1,2,3],[1,2,3],[1,2,3]])

In [9]:
x = ti.math.vec3([1,2,3])
y = ti.math.vec3([3,4,6])


x.outer_product(y).transpose()

[[ 3.  6.  9.]
 [ 4.  8. 12.]
 [ 6. 12. 18.]]

In [11]:
mesh = pv.read('1_0.vtm')['internal']

mesh = mesh.extract_cells([0,1,2,3,4,5])
m = Mesh(mesh)

In [31]:
m.cells

array([[ 1, 15, 22,  8,  0, 14, 21,  7],
       [ 2, 16, 23,  9,  1, 15, 22,  8],
       [ 3, 17, 24, 10,  2, 16, 23,  9],
       [ 4, 18, 25, 11,  3, 17, 24, 10],
       [ 5, 19, 26, 12,  4, 18, 25, 11],
       [ 6, 20, 27, 13,  5, 19, 26, 12]])

array([[ 5,  2,  1, 25, 19,  0],
       [ 8,  5,  4, 26, 20,  3],
       [11,  8,  7, 27, 21,  6],
       [14, 11, 10, 28, 22,  9],
       [17, 14, 13, 29, 23, 12],
       [18, 17, 16, 30, 24, 15]])

In [29]:
x = m.cell_centroids[0]

x,m.cells[0],m.faces.centroid[5]

(array([-24.875     ,  -9.66666651,   0.        ]),
 array([ 1, 15, 22,  8,  0, 14, 21,  7]),
 array([-24.75    ,  -9.666666,   0.      ], dtype=float32))

In [21]:
m.faces.cell_centroid_to_face_centroid[0,0]

array([1.25000000e-01, 4.76837158e-07, 0.00000000e+00])

In [30]:
m.faces.centroid[5] - x

array([1.25000000e-01, 4.76837158e-07, 0.00000000e+00])

In [13]:
m.set_boundary_value(0,u = 1,v = 2,w = 3.3,p = 2)

In [63]:
FVM = taichi_unstrucutred_mesh(m)

[Taichi] Starting on arch=x64


In [64]:
FVM.cell_centroids.shape,FVM.cell_centroids.n, FVM.cells.shape

((), 3, (6, 6))

TaichiIndexError: 
File "C:\Users\John Su\AppData\Local\Temp\ipykernel_5416\3284264371.py", line 126, in initial_face_interpolate:
            centroid_n = self.cell_centroids[C_n]
                         ^^^^^^^^^^^^^^^^^^^^^^^^
Field with dim 0 accessed with indices of dim 1